In [9]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
import time
import math
import re
import hashlib


In [13]:
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--start-maximized')
options.add_argument('--disable-infobars')  
options.add_argument('--disable-extensions')


service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

In [14]:
def dismiss_cookies(driver, timeout=8):
    wait = WebDriverWait(driver, timeout)
    tried = []

    def _visible(el):
        try:
            return el.is_displayed() and el.is_enabled()
        except Exception:
            return False

    # 0) Give the banner a second to mount
    driver.execute_script("window._probe = Date.now();")
    try:
        wait.until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "body")))
    except TimeoutException:
        pass

    # 1) Direct hit: common OneTrust IDs/classes
    candidates = [
        (By.ID, "onetrust-accept-btn-handler"),
        (By.CSS_SELECTOR, "button#onetrust-accept-btn-handler"),
        (By.CSS_SELECTOR,
         "#onetrust-banner-sdk button#onetrust-accept-btn-handler"),
        (By.CSS_SELECTOR, "button#onetrust-reject-all-handler"
         ),  # sometimes only Reject is visible first
        (By.CSS_SELECTOR, "[data-testid='onetrust-accept-btn-handler']"),
        (By.XPATH,
         "//button[contains(@id,'accept') and contains(translate(., 'ACEPT','acept'),'accept')]"
         ),
        (By.XPATH,
         "//button[contains(@aria-label,'Accept') or contains(normalize-space(.),'Accept')]"
         ),
    ]
    for how, what in candidates:
        tried.append(f"{how}={what}")
        try:
            btn = driver.find_element(how, what)
            if _visible(btn):
                btn.click()
                return True
        except NoSuchElementException:
            continue
        except WebDriverException:
            # Try JS click if the element exists but normal click fails
            try:
                driver.execute_script("arguments[0].click();", btn)
                return True
            except Exception:
                continue

    # 2) If not found, check for a banner container (present but hidden/animating)
    try:
        banner = driver.find_element(By.ID, "onetrust-banner-sdk")
        if _visible(banner):
            try:
                btn = banner.find_element(By.CSS_SELECTOR,
                                          "button[id*='accept']")
                driver.execute_script("arguments[0].click();", btn)
                return True
            except Exception:
                pass
    except NoSuchElementException:
        pass

    # 3)Scan iframes and try inside.
    iframes = driver.find_elements(By.TAG_NAME, "iframe")
    for i, frame in enumerate(iframes):
        # quick filter to avoid costly switches
        src = (frame.get_attribute("src") or "").lower()
        name = (frame.get_attribute("name") or "").lower()
        if any(k in src + name
               for k in ("consent", "onetrust", "privacy", "cookie")):
            tried.append(f"iframe[{i}] src={src or name}")
            try:
                driver.switch_to.frame(frame)
                # try common selectors again inside this frame
                for how, what in candidates:
                    try:
                        btn = WebDriverWait(driver, 2).until(
                            EC.presence_of_element_located((how, what)))
                        if _visible(btn):
                            driver.execute_script(
                                "arguments[0].scrollIntoView({block:'center'});",
                                btn)
                            try:
                                btn.click()
                            except Exception:
                                driver.execute_script("arguments[0].click();",
                                                      btn)
                            driver.switch_to.default_content()
                            return True
                    except Exception:
                        continue
                driver.switch_to.default_content()
            except Exception:
                # ensure we’re back
                try:
                    driver.switch_to.default_content()
                except:
                    pass

    # 4) Last resort: call OneTrust API if it exists, or remove the banner to unblock clicks
    try:
        ok = driver.execute_script("""
            if (window.OneTrust && OneTrust.AcceptAll) { OneTrust.AcceptAll(); return true; }
            const b = document.getElementById('onetrust-banner-sdk');
            if (b) { b.remove(); return 'removed'; }
            return false;
        """)
        if ok:
            return True
    except Exception:
        pass

    print("[cookies] Could not find/close cookie banner. Tried:",
          *tried,
          sep="\n - ")
    return False

In [15]:
url = 'https://www.mlssoccer.com/schedule/scores#competition=MLS-COM-000001&club=all'

driver.get(url)

dismiss_cookies(driver)
time.sleep(5) 
wait = WebDriverWait(driver, 10)

all_links = set()

rounds = 2000
stop_date = "Wednesday Oct 18, 2023"


for i in range(rounds):
    try:
        previous_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[aria-label="Previous results"]')))
        time.sleep(5)
        
        matches_table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'mls-c-schedule__matches')))
        if not matches_table:
            print("No matches table found on this page.")
            previous_button.click()
            continue
        
        date_headers = matches_table.find_elements(By.CSS_SELECTOR, 'h2.sc-hLBbgP.gIKMo')
        if date_headers:
            first_date_text = date_headers[0].text.strip()
            print("First date header:", first_date_text)
            if first_date_text == stop_date:
                print("Reached stop date.")
                break
        
        hrefs = matches_table.find_elements(By.TAG_NAME, 'a')
        if not hrefs:
            print("No match links found on this page.")
            previous_button.click()
            continue
        
        for href in hrefs:
            print(href.get_attribute('href'))
            all_links.add(href.get_attribute('href'))
            

        previous_button.click()

    except Exception as e:
        print(f"Error occurred: {e}")
        continue
print(f"Total unique match links collected: {len(all_links)}")

driver.quit()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=142.0.7444.59)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x4758c3
	0x475904
	0x28e76d
	0x26c32d
	0x3032ee
	0x31e799
	0x2fc766
	0x2cdac0
	0x2cede4
	0x6f7974
	0x6f2bea
	0x49e5b4
	0x48dd28
	0x494d8d
	0x47ded8
	0x47e09c
	0x467d1a
	0x75c75d49
	0x7765d6db
	0x7765d661


In [ ]:
"""all_links_df = pd.DataFrame(all_links, columns=['Match Links'])

all_links_df.to_csv('../data/mls_match_links.csv', index=False)"""

In [ ]:
links = pd.read_csv('../data/mls_match_links.csv')
links = links['Match Links'].tolist()

In [ ]:
def js_scroll_by(driver, dy):
    driver.execute_script("window.scrollBy(0, arguments[0]);", dy)



def js_scroll_into_view(driver, el):
    driver.execute_script(
        "arguments[0].scrollIntoView({block:'center', inline:'nearest'});", el)


def js_hover(driver, el):
    driver.execute_script(
        """
        const el = arguments[0];
        const rect = el.getBoundingClientRect();
        const x = Math.floor(rect.left + rect.width/2);
        const y = Math.floor(rect.top  + rect.height/2);
        ['pointerover','mouseover','mouseenter','mousemove'].forEach(type => {
            el.dispatchEvent(new MouseEvent(type, {bubbles:true, cancelable:true, clientX:x, clientY:y, view:window}));
        });
    """, el)
    
def scrape_cards(group):
    return driver.execute_script("""
        const root = arguments[0];
        return [...root.querySelectorAll('.mls-o-stat-chart')].map(c => ({
        stat:   c.querySelector('.mls-o-stat-chart__header')?.textContent.trim() || '',
        first:  c.querySelector('.mls-o-stat-chart__first-value')?.textContent.trim() || '',
        second: c.querySelector('.mls-o-stat-chart__second-value')?.textContent.trim() || '',
        }));
    """, group)
    
def make_match_id(link: str) -> str:
    mid = link.rstrip("/").split("/")[-1].split("?")[0]
    return re.sub(r"[^A-Za-z0-9._-]+", "_", mid)

In [ ]:
"""
FEED_DIR = Path('../data/scraping/matches/feed')


wait = WebDriverWait(driver, 6, poll_frequency=0.2)

for link in links:
    if (link is None or (isinstance(link, float) and math.isnan(link))
            or str(link).strip() == '' or str(link).strip().lower() == 'nan'):
        print(f"[skip] bad link: {link!r}")
        continue
    
    match_id = link.rstrip('/').split('/')[-1].split('?')[0]
    feed_path = FEED_DIR / f"feed_{match_id}.csv"

    if feed_path.exists():
        print(f"[skip] feed already done for {match_id}")
        continue

    feed = []


    try:
        driver.get(link)
        time.sleep(5)

        feed_bttn = driver.find_element(By.LINK_TEXT, 'Feed')

        try:
            feed_bttn.click()

            driver.execute_script(
                "window.scrollTo(0, (document.documentElement.scrollHeight - window.innerHeight) * 0.85);"
            )
            time.sleep(5)

            driver.execute_script(
                "window.scrollTo(0, (document.documentElement.scrollHeight - window.innerHeight) * 0.90);"
            )
            time.sleep(5)

            driver.execute_script(
                "window.scrollTo(0, (document.documentElement.scrollHeight - window.innerHeight) * 0.90);"
            )

            wait.until(
                EC.presence_of_element_located(
                    (By.XPATH,
                        "//*[normalize-space(text())='First half begins.']")))


            cont = driver.find_element(By.CSS_SELECTOR, 'div[class="mls-o-match-feed"]')

            for event in cont.find_elements(
                    By.XPATH,
                    './/div[contains(@class,"mls-o-match-feed__container")]'):
                minute_el = event.find_elements(By.CSS_SELECTOR, ".mls-o-match-feed__minute .mls-o-match-summary__regular-time")
                minute = minute_el[0].text.strip() if minute_el else None

                title_el = event.find_elements(By.CSS_SELECTOR, ".mls-o-match-feed__title")
                title = title_el[0].text.strip() if title_el else None

                comment_el = event.find_elements(By.CSS_SELECTOR, ".mls-o-match-feed__comment")
                comment = comment_el[0].text.strip() if comment_el else None

                feed.append({
                    'match_id': match_id,
                    'minute': minute,
                    'title': title,
                    'comment': comment
                })
        except Exception as e:
            print(f"Error processing feed for link {link}: {e}")
    except Exception as e:
        print(f"Error occurred while loading feed for link {link}: {e}")

    feed = pd.DataFrame(feed)

    general_stats = []
    shooting_stats = []
    passing_stats = []
    possession_stats = []
    xg_stats = []

    stats_bttn = driver.find_element(By.LINK_TEXT, 'Stats')

    try:
        stats_bttn.click()
        time.sleep(5)

        try:
            general_cont = wait.until(
                EC.presence_of_element_located((
                    By.XPATH,
                    '//section[contains(@class,"mls-l-module--stats-comparison")'
                    ' and contains(@class,"mls-l-module--general")'
                    ' and not(contains(@style,"display: none"))]')))


            js_scroll_into_view(driver, general_cont)
            general_cards = scrape_cards(general_cont)

            for it in general_cards:
                shooting_stats.append({
                    'stat_name': it['stat'],
                    'home_value': it['first'],
                    'away_value': it['second']
                })
        except Exception as e:
            print(f"Error occurred while scraping general stats: {e}")

        try:
            clubs_wrap = wait.until(
                EC.visibility_of_element_located((
                    By.XPATH,
                    '//section[contains(@class,"d3-l-section-row")][@data-toggle="clubs" and not(contains(@style,"display: none"))]'
                )))

            shooting_cont = clubs_wrap.find_element(
                By.XPATH,
                './/section[contains(@class,"mls-l-module--shooting-breakdown")]'
            )

            driver.execute_script(
                "arguments[0].scrollIntoView({block:'center'});",
                shooting_cont)

            shooting_cards = scrape_cards(shooting_cont)

            for it in shooting_cards:
                shooting_stats.append({
                    'stat_name': it['stat'],
                    'home_value': it['first'],
                    'away_value': it['second']
                })

        except Exception as e:
            print(f"Error occurred while scraping shooting stats: {e}")

        try:
            passing_cont = driver.find_element(By.XPATH, '//section[contains(@class,"passing-breakdown")]')

            passing_cards = scrape_cards(passing_cont)
            for it in passing_cards:
                passing_stats.append({
                    'stat_name': it['stat'],
                    'home_value': it['first'],
                    'away_value': it['second']
                })

        except Exception as e:
            print(f"Error occurred while scraping passing stats: {e}")

        try:
            possession_cont = driver.find_element(By.XPATH, '//section[contains(@class,"--possession")]')
            bar_cont = possession_cont.find_element(By.XPATH, './/*[contains(@class,"mls-o-possession__intervals")]')

            driver.execute_script(
                "arguments[0].scrollIntoView({block:'center'});",
                bar_cont)


            for bar in bar_cont.find_elements(By.XPATH, './/div[contains(@class,"mls-o-possession__average-intervals")]'):
                tip_id = bar.get_attribute('data-for')

                tooltips = bar.find_elements(By.XPATH, './/div[contains(@class,"__react_component_tooltip")]')

                tip = wait.until(EC.presence_of_element_located((By.ID, tip_id)))

                spans = tip.find_elements(By.XPATH, './/span')

                texts = [s.get_attribute('textContent').strip() for s in spans]
                texts = [t for t in texts if t and t.upper() != 'SKIP TO MAIN CONTENT']

                if len(texts) >= 4:
                    home_poss, home_adv, away_poss, away_adv = texts[:4]
                else:
                    home_poss = home_adv = away_poss = away_adv = None

                possession_stats.append({
                    'tip_id': tip_id,
                    'home_possession': home_poss,
                    'home_advantage': home_adv,
                    'away_possession': away_poss,
                    'away_advantage': away_adv
                })
        except Exception as e:
            print(f"Error occurred while scraping possession stats: {e}")

        try:
            xg_mod_xpath = (
                '//section[@data-toggle="clubs" and not(contains(@style,"display: none"))]'
                '//section[contains(@class,"mls-l-module--expected-goals")]'
            )
            xg_mod = wait.until(EC.visibility_of_element_located((By.XPATH, xg_mod_xpath)))

            groups = xg_mod.find_elements(
                By.CSS_SELECTOR,
                '.mls-o-expected-goals__chart-group, .mls-o-expected-goals__club-group'
            )
            chart_group = next(
                (g for g in groups if 'mls-o-expected-goals__chart-group' in (g.get_attribute('class') or '')),
                None
            )
            if chart_group is None:
                raise Exception("xG chart-group not found")

            # ensure cards exist
            wait.until(lambda d: any(
                e.is_displayed() for e in chart_group.find_elements(By.CSS_SELECTOR, '.mls-o-stat-chart')
            ))

            for card in chart_group.find_elements(By.CSS_SELECTOR, '.mls-o-stat-chart'):
                header = card.find_element(By.CSS_SELECTOR,  '.mls-o-stat-chart__header')
                first  = card.find_element(By.CSS_SELECTOR,  '.mls-o-stat-chart__first-value')
                second = card.find_element(By.CSS_SELECTOR,  '.mls-o-stat-chart__second-value')

                stat_name  = (header.text or header.get_attribute('textContent') or '').strip()
                home_value = (first.text  or first.get_attribute('textContent')  or '').strip()
                away_value = (second.text or second.get_attribute('textContent') or '').strip()

                xg_stats.append({
                    'stat_name': stat_name,
                    'home_value': home_value,
                    'away_value': away_value
                })
        except Exception as e:
            print(f"Error occurred while scraping expected goals stats: {e}")

    except Exception as e:
        print(f"Error occurred while scraping stats: {e}")
        pass

    player_rows = []
    gk_rows = []

    try:
        js_scroll_by(driver, -3000)
        time.sleep(2)

        player_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.mls-o-buttons__segment[value="players"]')))
        player_btn.click()


        players_mod = wait.until(EC.visibility_of_element_located((
            By.XPATH,
            '//section[contains(@class,"mls-l-module--match-hub-player-stats") and not(contains(@style,"display: none"))]'
        )))

        js_scroll_into_view(driver, players_mod)

        club_blocks = players_mod.find_elements(By.CSS_SELECTOR, '.mls-c-stats--match-hub-player-stats')

        for idx, block in enumerate(club_blocks):
            side = 'home' if idx == 0 else 'away'
            club_abbrev_el = block.find_elements(By.CSS_SELECTOR, '.mls-c-stats__club-abbreviation')
            club_abbrev = club_abbrev_el[0].text.strip() if club_abbrev_el else None

            for tbl in block.find_elements(By.CSS_SELECTOR, 'table.mls-o-table'):
                cls = (tbl.get_attribute('class') or '').lower()
                is_gk = 'goalkeeper' in cls

                header_cells = tbl.find_elements(
                    By.CSS_SELECTOR,
                    'thead .mls-o-table__header-group:not(.mls-o-table__header-group--main) .mls-o-table__header'
                )
                if not header_cells:
                    header_cells = tbl.find_elements(By.CSS_SELECTOR, 'thead .mls-o-table__header')

                headers = [(h.text or h.get_attribute('textContent') or '').strip() for h in header_cells]

                for tr in tbl.find_elements(By.CSS_SELECTOR, 'tbody .mls-o-table__row'):
                    cells = tr.find_elements(By.CSS_SELECTOR, '.mls-o-table__cell')
                    values = [(c.text or c.get_attribute('textContent') or '').strip() for c in cells]

                    if len(values) < len(headers):
                        values += [''] * (len(headers) - len(values))
                    elif len(values) > len(headers):
                        values = values[:len(headers)]

                    row = dict(zip(headers, values))
                    row.update({
                        'match_id': match_id,
                        'side': side,
                        'club': club_abbrev
                    })

                    if is_gk:
                        gk_rows.append(row)
                    else:
                        player_rows.append(row)

                    combined_rows = player_rows + gk_rows

    except Exception as e:
        print(f"Error occurred while scraping player stats: {e}")



    general_stats_df = pd.DataFrame(general_stats);  general_stats_df["category"] = "general"
    shooting_stats_df = pd.DataFrame(shooting_stats); shooting_stats_df["category"] = "shooting"
    passing_stats_df = pd.DataFrame(passing_stats);   passing_stats_df["category"] = "passing"
    possession_stats_df = pd.DataFrame(possession_stats); possession_stats_df["category"] = "possession"
    expected_goals_stats_df = pd.DataFrame(xg_stats); expected_goals_stats_df["category"] = "xg"
    player_stats_df = pd.DataFrame(combined_rows)

    all_stats = pd.concat(
        [general_stats_df, shooting_stats_df, passing_stats_df, possession_stats_df, expected_goals_stats_df],
        axis=0, ignore_index=True
    )

    feed.to_csv(f"../data/scraping/matches/feed/feed_{match_id}.csv", index=False)
    all_stats.to_csv(f"../data/scraping/matches/stats/stats_{match_id}.csv", index=False)
    player_stats_df.to_csv(f"../data/scraping/matches/players/player_stats_{match_id}.csv", index=False)

driver.quit()
"""

In [ ]:
wait = WebDriverWait(driver, 6, poll_frequency=0.2)


for link in links:
    if (link is None or (isinstance(link, float) and math.isnan(link))
            or str(link).strip() == '' or str(link).strip().lower() == 'nan'):
        print(f"[skip] bad link: {link!r}")
        continue

    match_id = link.rstrip('/').split('/')[-1].split('?')[0]

    driver.get(link)

    stats_bttn = wait.until(EC.visibility_of_element_located((By.LINK_TEXT, 'Stats')))


    stats_bttn.click()

    player_rows = []
    gk_rows = []

    try:
        js_scroll_by(driver, -3000)
        time.sleep(2)

        player_btn = wait.until(
            EC.element_to_be_clickable(
                (By.CSS_SELECTOR, '.mls-o-buttons__segment[value="players"]')))
        player_btn.click()

        players_mod = wait.until(
            EC.visibility_of_element_located((
                By.XPATH,
                '//section[contains(@class,"mls-l-module--match-hub-player-stats") and not(contains(@style,"display: none"))]'
            )))

        driver.execute_script("arguments[0].scrollIntoView({block:'center'});",
                            players_mod)

        abbrev_divs = players_mod.find_elements(
            By.CSS_SELECTOR, 'div.mls-c-stats__club-abbreviation')

        player_rows = []
        gk_rows = []

        for i, ab in enumerate(abbrev_divs):
            side = 'home' if i == 0 else 'away'
            club_abbrev = (ab.text or ab.get_attribute('textContent') or '').strip()

            # the table is the FIRST following sibling with class mls-c-stats__table
            tbl = ab.find_element(
                By.XPATH,
                'following-sibling::div[contains(@class,"mls-c-stats__table")][1]'
                '/table[contains(@class,"mls-o-table") and contains(@class,"match-hub-player-stats")]'
            )

            cls = (tbl.get_attribute('class') or '').lower()
            is_gk = 'goalkeeper' in cls

            header_cells = tbl.find_elements(
                By.CSS_SELECTOR,
                'thead .mls-o-table__header-group:not(.mls-o-table__header-group--main) .mls-o-table__header'
            ) or tbl.find_elements(By.CSS_SELECTOR, 'thead .mls-o-table__header')

            headers = [(h.text or h.get_attribute('textContent') or '').strip()
                    for h in header_cells]

            for tr in tbl.find_elements(By.CSS_SELECTOR, 'tbody .mls-o-table__row'):
                cells = tr.find_elements(By.CSS_SELECTOR, '.mls-o-table__cell')
                values = [(c.text or c.get_attribute('textContent') or '').strip()
                        for c in cells]

                if len(values) < len(headers):
                    values += [''] * (len(headers) - len(values))
                elif len(values) > len(headers):
                    values = values[:len(headers)]

                row = dict(zip(headers, values))
                row.update({'match_id': match_id, 'side': side, 'club': club_abbrev})

                if is_gk:
                    gk_rows.append(row)
                else: player_rows.append(row)

                combined_rows = player_rows + gk_rows

    except Exception as e:
        print(f"Error occurred while scraping player stats: {e}")

    player_stats_df = pd.DataFrame(combined_rows)
    player_stats_df.to_csv(
        f"../data/scraping/matches/players/player_stats_{match_id}.csv",
        index=False)
